## Introduction
This is an implementation to align the reads of a fasq file on the one chromosome "chr2L" of D melanogaster of a fasta file. <br>
To show how each part 's output is, there is a sample output of each part with a short description in the related output box. <br>
Please note that this work is for only reduced data set (mini datasets in ReadMe.pdf), that is, "dm6_chr2L.fasta" and "10k_reads.fastq"

In [1]:
import numpy as np
import sys
from collections import defaultdict
import timeit

## Read the "fasta" file

In [3]:
# read the chromosome (genome) and make its characters capital
start = timeit.default_timer()

with open("chr2L.fasta", 'r') as g:
    genome = ''
    for line in g.readlines()[1:]:
        genome += line.strip()
genome = genome.upper()

print(f"This is the first 100 nucleotides out of {len(genome)} nucleotides of the chr2L chromosome:")
print(genome[:100], '\n')
stop = timeit.default_timer()
print('Time (sec): ', round(stop - start, 3))

This is the first 100 nucleotides out of 23513712 nucleotides of the chr2L chromosome:
CGACAATGCACGACAGAGGAAGCAGAACAGATATTTAGATTGCCTCTCATTTTCTCTCCCATATTATAGGGAGAAATATGATCGCGTATGCGAGAGTAGT 

Time (sec):  14.262


## Construct suffix array
Please note that the performance of "function_suffix_array" function is tested by couple of different simple samples such as: <br>
'ATATA', 'banana', and/or 'Mississippi'

In [4]:
# this function is inspired from "http://algorithmicalley.com/archive/2013/06/30/suffix-arrays.aspx" 
start = timeit.default_timer()

def function_suffix_array(g, bucket, order=1):
    dict_bucket = defaultdict(list)
    for i in bucket:
        nucleotide = g[i: i+order]
        dict_bucket[nucleotide].append(i)
    suffix_array = list()
    for nt, indexList in sorted(dict_bucket.items()):
        if len(indexList) > 1:
            suffix_array += function_suffix_array(g, indexList, order*2)
        else:
            suffix_array.append(indexList[0])
    
    return suffix_array

suffix_array = function_suffix_array(genome, range(len(genome)))
print(f"the first 5 elements of the suffix array for the chr2L:\n {suffix_array[:5]}\n")

stop = timeit.default_timer()
print('Time (sec): ', round(stop - start, 3))

the first 5 elements of the suffix array for the chr2L:
 [10947977, 10947978, 10947979, 10947980, 10947981]

Time (sec):  87.577


## Read the "fastq" file

In [5]:
with open('10k_reads.fastq', 'r') as read:
    temp = read.readlines()
    list_reads = [r[:-1] for r in temp[1::4]]
print(f"the first 3 reads of the fastq file:\n{list_reads[:3]}")

the first 3 reads of the fastq file:
['TTCACCCTATAATATGGGAGAGAAAATGAGAGGCAA', 'TCTCGCATACGCGATCATATTTCACCCTATAATATG', 'TCTCGCATACGCGATCATATTTCACCCTATAATATG']


## Matching
Please note that the performance of "find_index" and/or "matching" function is tested by different simple samples and very famous methods like below: <br> 
looking for 'bc' in the string 'abccbcdfghhiiibc', using regular expression:

In [37]:
# this function is implemented based on binary search method
start = timeit.default_timer()

def find_index(suffix_array, read, genome):
    
    index_start = 0
    index_end = len(suffix_array)
    read_length = len(read)
    result = []
    
    while index_start <= index_end:
        index_mid = (index_start + index_end) // 2
        if genome[suffix_array[index_mid]: suffix_array[index_mid] + read_length] == read:
            result.append(suffix_array[index_mid])
            if genome[suffix_array[index_start]: suffix_array[index_start] + read_length] == read:
                result.extend(suffix_array[i] for i in range(index_start, index_mid))
            elif genome[suffix_array[index_end]: suffix_array[index_end] + read_length] == read:
                result.extend(suffix_array[i] for i in range(index_mid+1, index_end)) 
            elif genome[suffix_array[index_mid]: ] < read:
                for i in range(index_mid+1, index_end):
                    if genome[suffix_array[i]: suffix_array[i] + read_length] == read:
                        result.append(suffix_array[i])
            elif genome[suffix_array[index_mid]: ] > read:
                for i in range(index_start, index_mid):
                    if genome[suffix_array[i]: suffix_array[i] + read_length] == read:
                        result.append(suffix_array[i])
            break
            
        elif genome[suffix_array[index_mid]: ] < read: 
            index_start = index_mid + 1
        elif genome[suffix_array[index_mid]: ] > read: 
            index_end = index_mid - 1
            
    return sorted(result)

def matching(read):
    return find_index(suffix_array, read, genome)

index_200 = matching(list_reads[200])
print(f'the suffix array index of the read 200: {index_200}\n')

stop = timeit.default_timer()
print('Time (sec): ', round(stop - start, 3))

the suffix array index of the read 200: [12344]

Time (sec):  0.073


In [34]:
# check if the corresponding suffix of the index found by the function "matching" is really matched with the read
read_length = len(list_reads[0])
r = list_reads[200]
su = genome[12344: 12344 + read_length]
print(r)
print(su)
print('\nIs the read 200 matched with the corresponding suffix of the genome?', r == su)

TAAAACCATGATATCGCCCATATTCGTTAAACATAT
TAAAACCATGATATCGCCCATATTCGTTAAACATAT

Is the read 200 matched with the corresponding suffix of the genome? True


## Conclusion
This section is dedicated to how many reads of fastq file is mapped to fasta file 

In [35]:
start = timeit.default_timer()

counter = 0
for i in range(len(list_reads)):
    read = list_reads[i]
    list_idx = matching(read)
    if not list_idx: 
        counter += 1
print(f'the number of reads which are found at least one time in the fasta file: {counter}')
print(f'\nthe percentage of reads mapped = {round(counter/10000,2) * 100}')

stop = timeit.default_timer()
print('Time (sec): ', round(stop - start, 3))

the number of reads which are found at least one time in the fasta file: 6339

the percentage of reads mapped = 63.0
Time (sec):  983.023
